In [1]:
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, SpatialDropout1D

In [2]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df=pd.read_csv("spam_ham_dataset.csv")

In [4]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [5]:
df['text'] = df['text'].apply(lambda x:x.replace('\r\n',' '))

In [6]:
stemmer= PorterStemmer()
corpus = []
stopwords_set = set(stopwords.words('english'))
for i in range(len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans('','', string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

In [7]:
vectorizer = CountVectorizer()
x= vectorizer.fit_transform(corpus).toarray()
y=df.label_num
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [8]:
model = MultinomialNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')


Accuracy: 0.978743961352657
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       740
           1       0.96      0.96      0.96       295

    accuracy                           0.98      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.98      0.98      0.98      1035



In [9]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus)
X = pad_sequences(X, maxlen=100)


le = LabelEncoder()
y = le.fit_transform(df['label_num'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)


loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Accuracy: {accuracy}')

Epoch 1/5
65/65 - 23s - loss: 0.3489 - accuracy: 0.8225 - val_loss: 0.0944 - val_accuracy: 0.9662 - 23s/epoch - 351ms/step
Epoch 2/5
65/65 - 10s - loss: 0.0525 - accuracy: 0.9843 - val_loss: 0.0730 - val_accuracy: 0.9720 - 10s/epoch - 157ms/step
Epoch 3/5
65/65 - 9s - loss: 0.0187 - accuracy: 0.9942 - val_loss: 0.0524 - val_accuracy: 0.9836 - 9s/epoch - 143ms/step
Epoch 4/5
65/65 - 9s - loss: 0.0086 - accuracy: 0.9976 - val_loss: 0.0701 - val_accuracy: 0.9749 - 9s/epoch - 138ms/step
Epoch 5/5
65/65 - 8s - loss: 0.0041 - accuracy: 0.9993 - val_loss: 0.0606 - val_accuracy: 0.9778 - 8s/epoch - 121ms/step
33/33 - 0s - loss: 0.0606 - accuracy: 0.9778 - 418ms/epoch - 13ms/step
Accuracy: 0.9777777791023254


In [10]:
from tensorflow.keras.models import load_model
import pickle


In [11]:
# Save the trained model
model.save('spam_classifier_model.h5')

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
